In [3]:
import pandas as pd 
df= pd.read_csv('final_df.csv') 
print ("file loaded successfully!") 
display ( "\nlist of available columns:") 
print ( df.columns.tolist()) 

file loaded successfully!


'\nlist of available columns:'

['video_id', 'trending_date', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description', 'country', 'like_ratio', 'no_dislikes']


**Question 11: find the video with the largest number of views for each (trending_date, country) pair**

In [4]:

q11_result= df.loc[ df.groupby( ['trending_date' , 'country'])['views'].idxmax()] 
q11_result= q11_result.sort_values(by='views',ascending= False) 

print( "top videos per date and country:")
display(q11_result[['trending_date', 'country', 'title', 'views']].head())

top videos per date and country:


,trending_date,country,title,views
149926,18.07.04,United Kingdom,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,424538912
149726,18.06.04,United Kingdom,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,413586699
149522,18.05.04,United Kingdom,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,402650804
149325,18.04.04,United Kingdom,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,392036878
149130,18.03.04,United Kingdom,Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...,382401497


**Question 12: Divide trending_date into three columns: year,month, day**

In [10]:
df[ 'trending_date'] = pd.to_datetime( df['trending_date'] , format='%y.%d.%m')

df['year'] = df['trending_date'].dt.year
df['month'] = df['trending_date'].dt.month
df['day'] = df['trending_date'].dt.day

print( "columns created successfully!")
display(df[['trending_date', 'year', 'month', 'day']].head())

columns created successfully!


,trending_date,year,month,day
0,2017-11-14,2017,11,14
1,2017-11-14,2017,11,14
2,2017-11-14,2017,11,14
3,2017-11-14,2017,11,14
4,2017-11-14,2017,11,14


**Question 13: Find the video with the largest number of views for each (month,country) pair**

In [4]:

#we group by 'month' and 'country' ( created in q12 ), find the index of max views, and select those rows

q13_result= df.loc[ df.groupby( [ 'month','country']) ['views'].idxmax()] 
#sort the result by month to make it easier to read
q13_result= q13_result.sort_values(by=['month'])

print("top videos per month and country:")
display(q13_result[['month','country','title', 'views']])

top videos per month and country:


,month,country,title,views
11194,1,Canada,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,43067983
92416,1,France,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
51752,1,Germany,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
172246,1,India,"Taylor Swift - End Game ft. Ed Sheeran, Future",42019590
262517,1,Mexico,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,31680160
...,...,...,...,...
165675,12,India,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,125432237
256833,12,Mexico,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,100912384
296694,12,Russia,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,52611730
222104,12,South Korea,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,113876217


**Question 14: Read all json files from the 'data' folder**

In [5]:
import json
import os

folder_path = 'data'
category_data = {}

print(f"Scanning directory '{folder_path}' for JSON files\n")

try:
    all_files = os.listdir(folder_path)

    for filename in all_files:
        if filename.endswith('_category_id.json'):
            
            country_code = filename.split('_')[0]
            full_path = os.path.join(folder_path, filename)
            
            with open(full_path, 'r') as f:
                data = json.load(f)
                
                valid_ids = []
                for item in data['items']:
                    valid_ids.append(int(item['id']))
                
                category_data[country_code] = valid_ids
                print(f"✅ Loaded: {country_code} (from {filename})")

    print("\nFinished loading categories automatically.")

except FileNotFoundError:
    print(f"❌ Error: The folder '{folder_path}' was not found.")

Scanning directory 'data' for JSON files

✅ Loaded: IN (from IN_category_id.json)
✅ Loaded: RU (from RU_category_id.json)
✅ Loaded: JP (from JP_category_id.json)
✅ Loaded: FR (from FR_category_id.json)
✅ Loaded: GB (from GB_category_id.json)
✅ Loaded: KR (from KR_category_id.json)
✅ Loaded: MX (from MX_category_id.json)
✅ Loaded: CA (from CA_category_id.json)
✅ Loaded: US (from US_category_id.json)
✅ Loaded: DE (from DE_category_id.json)

Finished loading categories automatically.


**Question 15: Determine the number of videos with unassignable categories for each country**


In [6]:
# --- STRATEGY EXPLANATION ---
# The CSV file uses full country names (e.g., 'France'), while JSON filenames use 2-letter codes (e.g., 'FR').
#
# Note on Scalability & Choice of Method:
# For a large-scale global application, using a library like 'pycountry' would be the standard approach
# to automate name-to-code conversion.
# However, we opted for a manual mapping dictionary here for two reasons:
# 1. The dataset contains non-standard names (e.g., 'Great Britain' vs. ISO standard 'United Kingdom') which causes errors in automated libraries.
# 2. Since the scope is limited to only 10 countries, a manual dictionary ensures 100% accuracy without adding external dependencies.

country_mapper = {
    'United States' : 'US',
    'Great Britain' : 'GB',
    'United Kingdom' : 'GB' ,
    'Germany' : 'DE',
    'Canada' : 'CA',
    'France' : 'FR',
    'Russia' : 'RU',
    'Mexico' : 'MX',
    'South Korea': 'KR',
    'Japan' : 'JP',
    'India' : 'IN'
}

print (f"{'country' : <20} | {'Bad Categories Count'}") 
print ("_" * 45) 

for country_name in df [ 'country'] .unique():
    country_code = country_mapper.get(country_name)

# Check if we have the code and if we loaded its JSON data
    if country_code and country_code in category_data:
        
        valid_ids = category_data[country_code]
        country_df = df[df['country'] == country_name]
        
        # Count videos with category_id NOT in the valid list
        bad_videos = country_df[~country_df['category_id'].isin(valid_ids)]
        
        count = len(bad_videos)
        print(f"{country_name:<20} | {count}")
        
    else:
        print(f"{country_name:<20} | Data not found (Code: {country_code})")
    

country              | Bad Categories Count
_____________________________________________
Canada               | 74
Germany              | 242
France               | 111
United Kingdom       | 90
India                | 74
Japan                | 18
South Korea          | 286
Mexico               | 251
Russia               | 1538
United States        | 0
